### Association rules Personal Care
#### Department - personal care

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [5]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [6]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="personal care"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=1)

In [10]:
grouped_data

,order_id,product_name
0,98,Olive Oil & Aloe Vera Hand Soap
1,112,Umcka Elderberry Intensive Cold + Flu Berry Fl...
2,112,Premium Epsom Salt
3,226,Kids! Chewables Probiotic Tablets Natural Burs...
4,1143,Unscented Long Lasting Stick Deodorant
5,1143,Calming Lavender Body Wash
6,1572,Fluoride-Free Antiplaque & Whitening Peppermin...
7,1620,Rosemary Hand Soap
8,1994,Aquarium Pump Hand Soap
9,2415,Luden's Throat Drops Wild Cherry


In [11]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [12]:
#relace NAN with 0
table1=table1.fillna(0)

In [13]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [14]:
basket_sets = table1.applymap(encode_units)

In [41]:
## Running Apriori to get frequent itemset with a minimum support of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.001, use_colnames=True)

In [42]:
frequent_itemsets

,support,itemsets
0,0.002435,[18-in-1 Hemp Pure-Castile Soap Lavender]
1,0.001346,[2 in 1 Whitening Toothpaste]
2,0.001098,[24 Hour Non-Drowsy Allergy Relief]
3,0.001781,[3D White Mint Toothpaste]
4,0.002492,[3D White Radiant Mint Flavor Whitening Toothp...
5,0.001077,[Active Electrolyte Enhanced Drink Tabs 10 Ser...
6,0.001083,[Adult's Probiotic Capsules]
7,0.001301,[Advance White Fluoride Anticavity Tartar Cont...
8,0.001730,[Alcohol-Free Witch Hazel Rose Petal Toner]
9,0.001751,[All Natural Sea Fresh Antiplaque & Strengthen...


In [45]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [46]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Vegan Nutritional Shake Sweet Vanilla Bean),(Vegan Smooth Chocolate Nutritional Shake),0.003341,0.004108,0.001086,0.324955,79.095261,0.001072,1.475297
1,(Vegan Smooth Chocolate Nutritional Shake),(Vegan Nutritional Shake Sweet Vanilla Bean),0.004108,0.003341,0.001086,0.264234,79.095261,0.001072,1.354587
2,(Lemon Verbena Hand Soap),(Lavender Hand Soap),0.015420,0.018590,0.001397,0.090626,4.875059,0.001111,1.079215
3,(Lavender Hand Soap),(Lemon Verbena Hand Soap),0.018590,0.015420,0.001397,0.075173,4.875059,0.001111,1.064610
4,(Moroccan Argan Oil + Argan Stem Cell Triple M...,(Hair Shampoos),0.002789,0.002150,0.001014,0.363441,169.029400,0.001008,1.567568
5,(Hair Shampoos),(Moroccan Argan Oil + Argan Stem Cell Triple M...,0.002150,0.002789,0.001014,0.471409,169.029400,0.001008,1.886544


In [47]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Vegan Nutritional Shake Sweet Vanilla Bean),(Vegan Smooth Chocolate Nutritional Shake),0.003341,0.004108,0.001086,0.324955,79.095261,0.001072,1.475297
1,(Vegan Smooth Chocolate Nutritional Shake),(Vegan Nutritional Shake Sweet Vanilla Bean),0.004108,0.003341,0.001086,0.264234,79.095261,0.001072,1.354587
2,(Lemon Verbena Hand Soap),(Lavender Hand Soap),0.015420,0.018590,0.001397,0.090626,4.875059,0.001111,1.079215
3,(Lavender Hand Soap),(Lemon Verbena Hand Soap),0.018590,0.015420,0.001397,0.075173,4.875059,0.001111,1.064610
4,(Moroccan Argan Oil + Argan Stem Cell Triple M...,(Hair Shampoos),0.002789,0.002150,0.001014,0.363441,169.029400,0.001008,1.567568
5,(Hair Shampoos),(Moroccan Argan Oil + Argan Stem Cell Triple M...,0.002150,0.002789,0.001014,0.471409,169.029400,0.001008,1.886544


In [48]:
final_result.to_csv("Department_PersonalCareApriori.csv",sep=',')

In [49]:
test1=pd.read_csv("Department_PersonalCareApriori.csv")

In [50]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [51]:
test1=test1.replace({"u'": ''}, regex=True)

In [52]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [53]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [54]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [55]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Vegan Nutritional Shake Sweet Vanilla Bean,Vegan Smooth Chocolate Nutritional Shake,0.003341,0.004108,0.001086,0.324955,79.095261,0.001072,1.475297
1,Vegan Smooth Chocolate Nutritional Shake,Vegan Nutritional Shake Sweet Vanilla Bean,0.004108,0.003341,0.001086,0.264234,79.095261,0.001072,1.354587
2,Lemon Verbena Hand Soap,Lavender Hand Soap,0.015420,0.018590,0.001397,0.090626,4.875059,0.001111,1.079215
3,Lavender Hand Soap,Lemon Verbena Hand Soap,0.018590,0.015420,0.001397,0.075173,4.875059,0.001111,1.064610
4,Moroccan Argan Oil + Argan Stem Cell Triple Mo...,Hair Shampoos,0.002789,0.002150,0.001014,0.363441,169.029400,0.001008,1.567568
5,Hair Shampoos,Moroccan Argan Oil + Argan Stem Cell Triple Mo...,0.002150,0.002789,0.001014,0.471409,169.029400,0.001008,1.886544


In [56]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_PersonalCareApriori.csv")

In [57]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Vegan Nutritional Shake Sweet Vanilla Bean,Vegan Smooth Chocolate Nutritional Shake,0.003341,0.004108,0.001086,0.324955,79.095261,0.001072,1.475297
1,Vegan Smooth Chocolate Nutritional Shake,Vegan Nutritional Shake Sweet Vanilla Bean,0.004108,0.003341,0.001086,0.264234,79.095261,0.001072,1.354587
2,Lemon Verbena Hand Soap,Lavender Hand Soap,0.015420,0.018590,0.001397,0.090626,4.875059,0.001111,1.079215
3,Lavender Hand Soap,Lemon Verbena Hand Soap,0.018590,0.015420,0.001397,0.075173,4.875059,0.001111,1.064610
4,Moroccan Argan Oil + Argan Stem Cell Triple Mo...,Hair Shampoos,0.002789,0.002150,0.001014,0.363441,169.029400,0.001008,1.567568
5,Hair Shampoos,Moroccan Argan Oil + Argan Stem Cell Triple Mo...,0.002150,0.002789,0.001014,0.471409,169.029400,0.001008,1.886544


In [30]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()